In [1]:
import utils
import datetime
import pandas as pd
import instrument
import plotly.graph_objects as go

BUY = 1
SELL = -1

In [2]:
def create_candle_plot(df_plot, start=None, stop=None):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
        
    if start is not None:
        fig.add_vline(x=start, line_width=2, line_dash="dash", line_color="blue")
        
    if stop is not None:
        fig.add_vline(x=stop, line_width=2, line_dash="dash", line_color="blue")
    
    fig.update_layout(width=1000,height=300,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()
    

In [110]:
def get_date_as_string(df):
    return df.loc[(len(df.index) - 1), "time"].strftime("%Y-%m-%d")


def get_price_data(pairname, granularity):
    df = pd.read_pickle(utils.get_his_data_filename(pairname, granularity))
    non_cols = ["time", "volume"]
    mod_cols = [x for x in df.columns if x not in non_cols]
    df[mod_cols] = df[mod_cols].apply(pd.to_numeric)
    return df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]]


def get_day_data(date, price_data):
    start_time = date
    end_time = date + datetime.timedelta(days=1)
    df_day = price_data[
        (price_data.time >= start_time) & (price_data.time < end_time)
    ].copy()
    df_day.dropna(inplace=True)
    df_day.reset_index(drop=True, inplace=True)
    return df_day


def get_post_london_open_data(df):
    day = get_date_as_string(df)
    london_start_time = utils.get_utc_dt_from_string(f"{day} 07:00:00+00:00")
    df_post_london_open = df[(df.time >= london_start_time)].copy()
    df_post_london_open.reset_index(drop=True, inplace=True)
    return df_post_london_open


def calculate_london_start_time(day):
    return utils.get_utc_dt_from_string(f"{day} 07:00:00+00:00")


def calculate_tokyo_start_time(day):
    return utils.get_utc_dt_from_string(f"{day} 01:00:00+00:00")


def calculate_pre_open_range(df_day):
    date = utils.get_utc_dt_from_string(get_date_as_string(df_day))
    tokyo_open_time = calculate_tokyo_start_time(date)
    df_pre_tokyo_open = df_day[(df_day.time < (tokyo_open_time - datetime.timedelta(hours=1)))].copy()
    range_max = max(df_pre_tokyo_open.mid_h.max(), df_pre_tokyo_open.mid_l.max())
    range_min = min(df_pre_tokyo_open.mid_h.min(), df_pre_tokyo_open.mid_l.min())
    return range_min, range_max


def calculate_take_profit(position_type, open_price, pip_position=0.0001):
    if position_type == BUY:
        take_profit = open_price + (pip_position)*90
    elif position_type == SELL:
        take_profit = open_price - (pip_position)*90
    return take_profit    


def calculate_stop_loss(position_type, open_price,  pip_position=0.0001):
    if position_type == BUY:
        stop_loss = open_price - (pip_position)*30
        
    elif position_type == SELL:
        stop_loss = open_price + (pip_position)*30
    return stop_loss

In [95]:
def get_close_time_as_string(df):
    close_time = df.loc[(len(df.index) - 1), "time"].strftime("%H:%M:%S")
    return close_time

def exit_time(df):
    return utils.get_utc_dt_from_string(
        f"{get_date_as_string(df)} {get_close_time_as_string(df)}"
    )

def count_open_trades(df):
    buy_list = [x for x in df.IS_TRADE if x == BUY]
    sell_list = [x for x in df.IS_TRADE if x == SELL]
    buys = len(buy_list)
    sells = len(sell_list)
    return {"total": (buys + sells), "buys": buys, "sells": sells}

def over_range_max(row, range_max):
    return row.mid_c > range_max

def under_range_min(row, range_min):
    return row.mid_c < range_min

def get_past_data(df, index):
    return df.iloc[:index].copy()

def over_take_profit(row, take_profit):
    return row.mid_c > take_profit

def over_stop_loss(row, stop_loss):
    return row.mid_c > stop_loss

def under_take_profit(row, take_profit):
    return row.mid_c < take_profit

def under_stop_loss(row, stop_loss):
    return row.mid_c < stop_loss

def get_first_check_time(df):
    date = get_date_as_string(df)
    return utils.get_utc_dt_from_string(f"{date} 04:15:00+00:00")

def get_second_check_time(df):
    date = get_date_as_string(df)
    return utils.get_utc_dt_from_string(f"{date} 08:15:00+00:00")

def calculate_trades(df):
    
    print(df.head())
    
    price_at_t1 = df[df['time'] == get_first_check_time(df)].mid_c.tolist().pop()
    price_at_t2 = df[df['time'] == get_second_check_time(df)].mid_c.tolist().pop()
    open_trade_index = (df[df['time'] == get_second_check_time(df)].index.tolist().pop() + 1) 
    open_trade_price = df.loc[open_trade_index, "mid_c"]   
    
    df["IS_TRADE"] = 0
    if (price_at_t2 - price_at_t1) <= 0:
        df.loc[open_trade_index, "IS_TRADE"] = BUY
    elif (price_at_t2 - price_at_t1) > 0:
        df.loc[open_trade_index, "IS_TRADE"] = SELL
    df = df[df.index >= open_trade_index]
    df.reset_index(drop=True, inplace=True)

    # print(open_trade_price)
    # print(calculate_stop_loss(BUY, open_trade_price))
    
    for index, row in df.iterrows():
        trade_count = count_open_trades(get_past_data(df, index))
        if trade_count["total"] == 1:
            if trade_count["buys"] == 1:
                if over_take_profit(row, calculate_take_profit(BUY, open_trade_price)):
                    df.loc[index, "IS_TRADE"] = SELL           
                elif under_stop_loss(row, calculate_stop_loss(BUY, open_trade_price)):
                    df.loc[index, "IS_TRADE"] = SELL
            elif trade_count["sells"] == 1:
                if under_take_profit(row, calculate_take_profit(SELL, open_trade_price)):
                    df.loc[index, "IS_TRADE"] = BUY
                elif over_stop_loss(row, calculate_stop_loss(SELL, open_trade_price)):
                    df.loc[index, "IS_TRADE"] = BUY
        if row.time >= exit_time(df) and trade_count['total'] == 1:
            if trade_count["buys"] == 1:
                df.loc[index, "IS_TRADE"] = SELL
            if trade_count["sells"] == 1:
                df.loc[index, "IS_TRADE"] = BUY
    return df

def process_trades(df, i_pair):
    df_trades = df[df.IS_TRADE != 0].copy()
    df_trades["DELTA"] = df_trades.mid_c.diff() / i_pair.pipLocation
    df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"] * (-1)
    return df_trades

In [105]:
def test_trade(date, pairname, granularity):
    price_data = get_price_data(pairname, granularity)
    i_pair = instrument.Instrument.get_instruments_dict()[pairname]
    date = utils.get_utc_dt_from_string(date)
    df = get_day_data(date, price_data) 
    create_candle_plot(
        df,
        start= utils.get_utc_dt_from_string(f"{date} 04:15:00+00:00"), 
        stop =utils.get_utc_dt_from_string(f"{date} 08:15:00+00:00") 
    )
    df = calculate_trades(df)
    df_trades = process_trades(df, i_pair)
    return df_trades


def is_weekend(date):
    if date.weekday() >= 5:
        return True
    return False


def run_pair(start_date, end_date, pairname, granularity):
    price_data = get_price_data(pairname, granularity)
    i_pair = instrument.Instrument.get_instruments_dict()[pairname]
    start_date = utils.get_utc_dt_from_string(start_date)
    end_date = utils.get_utc_dt_from_string(end_date)
    day_count = (end_date - start_date).days - 1
    days = []
    results = []
    for date in (start_date + datetime.timedelta(n) for n in range(day_count)):
        days.append(date)
        df = get_day_data(date, price_data) 
        if df.empty or is_weekend(date):
            results.append(0.0)
        else:
            df = calculate_trades(df)
            df_trades = process_trades(df, i_pair)
            if len(df_trades.IS_TRADE.values.tolist()) == 2:
                gain = round(df_trades["GAIN"].iloc[-1], 5)
            elif len(df_trades.IS_TRADE.values.tolist()) == 0:
                gain = 0
            results.append(gain)
    df_results = pd.DataFrame({"date": days, "pips": results})
    df_results["gain"] = df_results.pips.cumsum()
    return df_results

In [113]:
granularity = "M15"
date = "2018-12-25"

print(is_weekend(utils.get_utc_dt_from_string(date)))

pairname = 'GBP_USD'
df_trades = test_trade(date, pairname, granularity)

df_trades.head()

False


                       time    mid_o    mid_h    mid_l    mid_c
0 2018-12-25 22:00:00+00:00  1.27143  1.27143  1.26709  1.26740
1 2018-12-25 22:15:00+00:00  1.26740  1.26760  1.26731  1.26733
2 2018-12-25 22:30:00+00:00  1.26757  1.26867  1.26725  1.26749
3 2018-12-25 22:45:00+00:00  1.26744  1.26884  1.26726  1.26768
4 2018-12-25 23:00:00+00:00  1.26735  1.26918  1.26726  1.26917


IndexError: pop from empty list

In [111]:
granularity = "M15"
start_date = "2018-01-02"
end_date = "2019-01-02"

pairs = ['GBP_USD']

fig = go.Figure()
for pairname in pairs:
    df_results = run_pair(start_date, end_date, pairname, granularity)
    fig.add_trace(
        go.Scatter(
            x=df_results.date,
            y=df_results.gain,
            line=dict(width=2),
            line_shape="linear",
            name=pairname,
        )
    )
fig.show()


                       time    mid_o    mid_h    mid_l    mid_c
0 2018-01-02 00:00:00+00:00  1.35105  1.35138  1.35082  1.35114
1 2018-01-02 00:15:00+00:00  1.35115  1.35162  1.35114  1.35146
2 2018-01-02 00:30:00+00:00  1.35146  1.35194  1.35136  1.35162
3 2018-01-02 00:45:00+00:00  1.35162  1.35163  1.35101  1.35126
4 2018-01-02 01:00:00+00:00  1.35128  1.35220  1.35121  1.35213
1.35532
1.3523200000000002
                       time    mid_o    mid_h    mid_l    mid_c
0 2018-01-03 00:00:00+00:00  1.35946  1.35994  1.35942  1.35966
1 2018-01-03 00:15:00+00:00  1.35958  1.35978  1.35936  1.35952
2 2018-01-03 00:30:00+00:00  1.35950  1.35952  1.35888  1.35910
3 2018-01-03 00:45:00+00:00  1.35908  1.35960  1.35908  1.35960
4 2018-01-03 01:00:00+00:00  1.35958  1.36010  1.35948  1.35964
1.35993
1.3569300000000002
                       time    mid_o    mid_h    mid_l    mid_c
0 2018-01-04 00:00:00+00:00  1.35106  1.35108  1.35056  1.35065
1 2018-01-04 00:15:00+00:00  1.35066  1.35110  1.3

IndexError: pop from empty list